# Experiment 6

- try `crossvalidation` for the best models
- use normalisation (`SVC` need normalisation 'exp1')


In [11]:
import time 
import pandas as pd
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,GradientBoostingClassifier
from sklearn .svm import SVC
from sklearn .metrics import roc_auc_score,accuracy_score,confusion_matrix,f1_score
from xgboost import XGBClassifier
import json
import numpy as np
from sklearn.preprocessing import StandardScaler




In [7]:
df = pd.read_csv("../data/cleaned_data.csv")
with open('../result/xgbc_best_hyperparameters.json') as f :
    xgbc_best_params=json.load(f)
with open('../result/rfc_best_hyperparameters.json') as f :
    rfc_best_params=json.load(f)
with open('../result/svc_best_hyperparameters.json') as f :
    svc_best_params=json.load(f)
with open('../result/gbc_best_hyperparameters.json') as f :
    gbc_best_params=json.load(f)

In [36]:
def evaluate_model(model,model_name:str,train:tuple,test:tuple,cv=10):
    start_train=time.time()
    model.fit(train[0],train[1])
    end_train=time.time()
    start_test=time.time()
    preds = model.predict(test[0])
    end_test=time.time()
    accuracy=accuracy_score(test[1],preds)
    f1_=f1_score(test[1],preds)
    auc=roc_auc_score(test[1],preds)
    cm=confusion_matrix(test[1],preds)
    return {"model":model_name,
           "accuracy":accuracy,
            "auc":auc,
            'f1_score':f1_,
            "cm":cm,
            'training_time(ms)':1000*(end_train-start_train),
            'testing_time(ms)':1000*(end_test-start_test)
           }

In [37]:
models=[
    XGBClassifier(**xgbc_best_params),
    RandomForestClassifier(**rfc_best_params),
    VotingClassifier([
        ('xgb',XGBClassifier(**xgbc_best_params)),
        ('rfc',RandomForestClassifier(**rfc_best_params))
    ],voting='soft'),
    SVC(**svc_best_params,probability=True),
    GradientBoostingClassifier(**gbc_best_params)
]

In [28]:
df=pd.read_csv('../data/cleaned_data.csv')
X_train,X_test,y_train,y_test=train_test_split(df.drop('target',axis=1),df.target,test_size=0.2,random_state=42)
scaler=StandardScaler()
train=(scaler.fit_transform(X_train),y_train)
test=(scaler.transform(X_test),y_test)

In [38]:
def evaluate_models(models,train,test,cv=10):
    result = []
    for model in models:
        scores=cross_val_score(model,train[0],train[1],cv=cv,scoring='roc_auc_ovo')
        rs=evaluate_model(model,model.__class__.__name__,train,test)
        rs.update({
            'scoring':scores,
            'mean_scores':scores.mean(),
            'std_scores':scores.std()
        })
        result.append(rs)
    return pd.DataFrame(result) 

In [39]:
rseult = evaluate_models(models,train,test)

In [49]:
rseult.to_csv('../result/exp6.csv',index=False)